In [1]:
import findspark
findspark.init('/home/asif/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MovieReco').getOrCreate()

## This dataset is too large to include here . the link is : [Movie lens dataset](https://grouplens.org/datasets/movielens/)

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [7]:
data = spark.read.csv('ratings.csv',inferSchema=True,header=True)

In [8]:
data.count()

20000263

In [9]:
data.describe().show()

+-------+-----------------+------------------+------------------+--------------------+
|summary|           userId|           movieId|            rating|           timestamp|
+-------+-----------------+------------------+------------------+--------------------+
|  count|         20000263|          20000263|          20000263|            20000263|
|   mean|69045.87258292554| 9041.567330339605|3.5255285642993797|1.1009179216771157E9|
| stddev|40038.62665316201|19789.477445413086| 1.051988919294227|1.6216942478273067E8|
|    min|                1|                 1|               0.5|           789652004|
|    max|           138493|            131262|               5.0|          1427784002|
+-------+-----------------+------------------+------------------+--------------------+



In [16]:
training, test = data.randomSplit([0.95,0.05])

In [17]:
als = ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [18]:
trained_model = als.fit(training)

In [25]:
predictions = trained_model.transform(test)

In [20]:
predictions.show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
| 74757|    148|   3.5|1064853335|  3.091398|
| 92852|    148|   3.0| 839813031| 2.4858449|
| 36821|    148|   4.0| 979367598| 2.6552389|
| 83090|    148|   2.0|1030400425| 2.1114237|
|  1259|    148|   5.0| 828625487|  2.932781|
| 54726|    148|   5.0| 832703670| 3.6909611|
| 90757|    148|   3.0| 970000570| 3.4056427|
| 81300|    148|   1.0| 983118964| 2.8901615|
| 68242|    148|   3.0|1047397251| 2.4199088|
|109910|    148|   2.0| 907093395|  2.107089|
| 70446|    148|   2.0| 945907892| 2.7358654|
| 44926|    148|   2.0| 839943770|  2.791694|
|104919|    148|   3.0| 857133054| 3.6221476|
| 36200|    148|   3.0| 828231965| 3.3647413|
| 73654|    148|   4.0| 833509066| 3.6824224|
|107346|    148|   3.0| 853421750| 2.5520804|
|132466|    148|   3.0| 945399307| 2.7650256|
| 91723|    148|   4.0| 828042393| 3.2435424|
| 59340|    148|   4.0|1034547175|

In [26]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [27]:
rmse =  evaluator.evaluate(predictions)
print('RMSE=',rmse)


RMSE= nan


In [28]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|    380|    11|
|    595|    11|
|    736|    11|
|    741|    11|
|   1136|    11|
|   3159|    11|
|   3438|    11|
|   4105|    11|
|   5378|    11|
|   6711|    11|
|   6874|    11|
|   7439|    11|
|   8907|    11|
|  30707|    11|
|  33679|    11|
|  36401|    11|
|  43928|    11|
|  43936|    11|
|  44225|    11|
|  44731|    11|
+-------+------+
only showing top 20 rows



In [29]:
recomendations = trained_model.transform(single_user)
recomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|    595|    11| 4.5475717|
|   6874|    11| 4.3705063|
|  56174|    11|  4.257205|
|  56757|    11| 4.2407002|
|   5378|    11|  4.226343|
|   1136|    11|  4.225267|
|  30707|    11| 4.1249785|
|  33679|    11|  4.041348|
|   3159|    11|  4.010023|
|    741|    11| 3.9870813|
|  58025|    11| 3.8815458|
|  36401|    11| 3.8535488|
|  43936|    11|  3.837541|
|    736|    11| 3.8061619|
|    380|    11| 3.7669373|
|   6711|    11| 3.7245393|
|   8907|    11| 3.7160368|
|   3438|    11| 3.7133937|
|  64497|    11|  3.666827|
|   7439|    11| 3.5691798|
+-------+------+----------+
only showing top 20 rows

